In [ ]:
# Imports

In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
from src.utils.preprocess import load_isic_data, get_data_generators, compute_class_weights
from src.model.model import build_model, compile_model, fine_tune_model, get_gradcam_heatmap

In [ ]:
# Set paths

In [ ]:
data_dir = 'data/ISIC2018_Task3_Training_Input'
csv_file = 'data/ISIC2018_Task3_Training_GroundTruth.csv'

In [ ]:
# load data

In [ ]:
image_paths, labels, class_names = load_isic_data(data_dir, csv_file)
print(f'Classes: {class_names}')
print(f'Total images: {len(image_paths)}')

In [ ]:
# Plot class distribution

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(x=labels)
plt.xticks(ticks=range(len(class_names)), labels=class_names, rotation=45)
plt.title('Class Distribution')
plt.show()

In [ ]:
# Get data generators

In [ ]:
train_generator, val_generator, class_names = get_data_generators(data_dir, csv_file)

In [ ]:
# Compute class weights

In [ ]:
class_weights = compute_class_weights(labels)

In [ ]:
# Build and compile model

In [ ]:
model = build_model(num_classes=len(class_names))
model = compile_model(model)
model.summary()

In [ ]:
# Train model

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint('skin_lesion_model.h5', save_best_only=True)
]

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    class_weight=class_weights,
    callbacks=callbacks
)

In [ ]:
# Fine-tune model

In [ ]:
model = fine_tune_model(model, trainable_layers=10)
model = compile_model(model, learning_rate=1e-5)
history_fine = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    class_weight=class_weights,
    callbacks=callbacks
)

In [ ]:
# Evaluate model

In [ ]:
val_preds = model.predict(val_generator)
val_preds_classes = np.argmax(val_preds, axis=1)
val_true = val_generator.classes

In [ ]:
# Confusion matrix

In [ ]:
cm = confusion_matrix(val_true, val_preds_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Classification report
print(classification_report(val_true, val_preds_classes, target_names=class_names))

In [ ]:
# Plot training history

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.legend()
plt.title('Accuracy')
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.title('Loss')
plt.show()

In [ ]:
# Example Grad-CAM

In [ ]:
sample_img, _ = next(val_generator)
heatmap = get_gradcam_heatmap(model, sample_img[:1])
plt.imshow(sample_img[0])
plt.imshow(heatmap, cmap='jet', alpha=0.5)
plt.title('Grad-CAM Heatmap')
plt.show()